In [7]:
import pandas as pd

def select_non(series):
    return series.dropna().iloc[0] if not series.dropna().empty else None

def preprocess(file_path, item_names_to_extract):
    # Load the data
    data = pd.read_excel(file_path)

    # Erase blank spaces and special characters in '항목명'
    data["항목명"] = (
        data["항목명"].str.replace(" ", "").str.replace(r"[^가-힣]", "", regex=True)
    )

    # If '종목코드' column is present, format it by removing brackets
    if "종목코드" in data.columns:
        data["종목코드"] = data["종목코드"].str.replace("[", "").str.replace("]", "")

    # Extract the specified item names
    extracted_data = data[data["항목명"].isin(item_names_to_extract)]
    grouped_data = extracted_data.groupby(['회사명', '종목코드', '항목명']).agg({'당기': select_non}).reset_index()

    final_data = grouped_data.pivot(
        index=["회사명", "종목코드"], columns="항목명", values="당기"
    ).reset_index()

    return final_data



# Example usage:
# df_financial_data = process_financial_excel('path_to_financial_data.xlsx', ['유동자산', '비유동자산', ...])

BS_item = ["유동자산", "비유동자산", "자산총계", "자본총계", "부채및자본총계", "유동부채", "비유동부채", "부채총계"]
CF_item = ["영업활동현금흐름"]
PL_item = ["매출액", "매출총이익", "순이익"]


def fillnan(df):
    # Filling NaN values using the provided formulas
    df["자산총계"] = df["자산총계"].fillna(df["유동자산"] + df["비유동자산"])
    df["부채총계"] = df["부채총계"].fillna(df["유동부채"] + df["비유동부채"])
    df["부채및자본총계"] = df["부채및자본총계"].fillna(df["부채총계"] + df["자본총계"])
    return df

In [3]:
def fillnan(df):
    # Filling NaN values using the provided formulas
    df["자산총계"] = df["자산총계"].fillna(df["유동자산"] + df["비유동자산"])
    df["부채총계"] = df["부채총계"].fillna(df["유동부채"] + df["비유동부채"])
    df["부채및자본총계"] = df["부채및자본총계"].fillna(df["부채총계"] + df["자본총계"])
    df["자본총계"] = df["자본총계"].fillna(df["부채및자본총계"] - df["부채총계"])
    return df

In [24]:
BS_23_1Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2023_1Q_BS_filter.xlsx", BS_item)
CF_23_1Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2023_1Q_CF_filter.xlsx", CF_item)
PL_23_1Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2023_1Q_PL_filter.xlsx", PL_item)

In [26]:
df_23_1Q = pd.merge(BS_23_1Q, CF_23_1Q, on=['회사명', '종목코드'], how='outer')
df_23_1Q = pd.merge(df_23_1Q, PL_23_1Q, on=['회사명', '종목코드'], how='outer')

In [10]:
label_23_1Q = pd.read_excel("C:/Users/dochy/Desktop/고려대/딥러닝/project/label_23_1Q.xlsx")

c:\Users\dochy\anaconda3\envs\dl\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [12]:
import pandas as pd
drop_columns = ['시작일 기준가','종료일 종가','대비','거래량','거래대금']

def create_class_and_merge(data_df, label_df, drop_columns):
    
    # 클래스 생성
    label_df['class'] = label_df['등락률'].apply(lambda x: 0 if x <= -10 else (1 if x < 0 else (2 if x < 10 else 3)))

    # 불필요한 열 제거
    label_df = label_df.drop(drop_columns, axis=1)

    # 데이터프레임 병합
    merged_df = pd.merge(data_df, label_df, on=['종목코드'], how='outer')

    return merged_df



In [27]:
df_1Q_2023 = create_class_and_merge(df_23_1Q, label_23_1Q, drop_columns)
df_1Q_2023 = fillnan(df_1Q_2023)
df_1Q_2023.to_csv('df_23_1Q')

In [29]:
BS_20_1Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_1Q_BS_filter.xlsx", BS_item)
CF_20_1Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_1Q_CF_filter.xlsx", CF_item)
PL_20_1Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_1Q_PL_filter.xlsx", PL_item)

In [31]:
df_20_1Q = pd.merge(BS_20_1Q, CF_20_1Q, on=['회사명', '종목코드'], how='outer')
df_20_1Q = pd.merge(df_20_1Q, PL_20_1Q, on=['회사명', '종목코드'], how='outer')

In [32]:
label_20_1Q = pd.read_excel("C:/Users/dochy/Desktop/고려대/딥러닝/project/label_20_1Q.xlsx")

c:\Users\dochy\anaconda3\envs\dl\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [38]:
df_1Q_2020 = create_class_and_merge(df_20_1Q, label_20_1Q, drop_columns)
df_1Q_2020 = fillnan(df_1Q_2020)
df_1Q_2020.to_csv('df_20_1Q')

In [35]:
BS_20_2Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_2Q_BS_filter.xlsx", BS_item)
CF_20_2Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_2Q_CF_filter.xlsx", CF_item)
PL_20_2Q = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_2Q_PL_filter.xlsx", PL_item)

In [36]:
df_20_2Q = pd.merge(BS_20_2Q, CF_20_2Q, on=['회사명', '종목코드'], how='outer')
df_20_2Q = pd.merge(df_20_2Q, PL_20_2Q, on=['회사명', '종목코드'], how='outer')

In [37]:
label_20_2Q = pd.read_excel("C:/Users/dochy/Desktop/고려대/딥러닝/project/label_20_2Q.xlsx")

c:\Users\dochy\anaconda3\envs\dl\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [39]:
df_2Q_2020 = create_class_and_merge(df_20_2Q, label_20_2Q, drop_columns)
df_2Q_2020 = fillnan(df_2Q_2020)
df_2Q_2020.to_csv('df_20_1Q')

In [57]:
df_test = pd.concat([df_1Q_2023,df_2Q_2020],axis=0)
df_test = pd.concat([df_test,df_1Q_2020],axis=0)

In [65]:
df_test.head()

,회사명,종목코드,부채및자본총계,부채총계,비유동부채,비유동자산,유동부채,유동자산,자본총계,자산총계,영업활동현금흐름,매출액,매출총이익,순이익,종목명,등락률,class
0,3S,060310,6.595678e+10,2.006824e+10,2.424885e+09,4.119151e+10,1.764336e+10,2.476527e+10,4.588854e+10,6.595678e+10,5.641026e+09,1.331206e+10,1.920551e+09,1.081571e+09,3S,23.63,3.0
1,AJ네트웍스,095570,1.390079e+12,1.001349e+12,2.680423e+11,1.186349e+12,7.333065e+11,2.037299e+11,NaN,1.390079e+12,3.549927e+09,NaN,NaN,4.755168e+09,AJ네트웍스,-4.88,1.0
2,AK홀딩스,006840,9.322353e+11,3.811836e+11,2.081565e+07,8.567521e+11,3.811627e+11,7.548319e+10,5.510517e+11,9.322353e+11,1.502585e+11,1.102303e+12,2.968355e+11,6.843604e+10,AK홀딩스,8.61,2.0
3,APS,054620,2.753778e+11,9.357518e+10,2.270707e+10,2.245058e+11,7.086811e+10,5.087194e+10,1.818026e+11,2.753778e+11,-4.631668e+09,4.998406e+09,2.104061e+09,-3.391231e+09,APS,-31.71,0.0
4,AP시스템,265520,5.436351e+11,2.671041e+11,5.505503e+10,1.513681e+11,2.120491e+11,3.922670e+11,2.765309e+11,5.436351e+11,-4.762685e+10,9.397471e+10,2.659035e+10,1.218080e+10,AP시스템,-4.26,1.0


In [69]:
input_layer = len(df_test.columns)
drop_columns_test =['회사명', '종목코드', '종목명' , '등락률']

In [80]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn


test = df_test.drop(drop_columns_test,axis=1)
test.dropna(inplace=True)
X = test.drop('class', axis=1)
y = test['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [87]:
y_train

tensor([1., 3., 3.,  ..., 3., 0., 3.])

In [85]:
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        
        self.fc1 = nn.Linear(12, 64)  # Adjust input size to 3
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [86]:
import torch.optim as optim
# Create the DNN model
model = DNNModel()

#Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    loss.backward()
    optimizer.step()

# Evaluate the model on the test set
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = (y_pred >= 0.5).float()  # Convert probabilities to binary predictions

    accuracy = accuracy_score(y_test.numpy(), y_pred.numpy())
    print(f'Accuracy: {accuracy:.2f}')

# 실패한 label

RuntimeError: all elements of target should be between 0 and 1

In [82]:
import pandas as pd
df_2Q_2020 = pd.read_csv('C:/Users/dochy/Desktop/고려대/딥러닝/project/df_2Q_2022',dtype={'종목코드': str})
df_1Q_2020 = pd.read_csv('C:/Users/dochy/Desktop/고려대/딥러닝/project/df_1Q_2020',dtype={'종목코드': str})

In [51]:
df_1Q_2020_label = pd.read_excel('C:/Users/dochy/Desktop/고려대/딥러닝/project/label_20_1Q.xlsx')
df_2Q_2020_label = pd.read_excel('C:/Users/dochy/Desktop/고려대/딥러닝/project/label_20_2Q.xlsx')


c:\Users\dochy\anaconda3\envs\dl\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\dochy\anaconda3\envs\dl\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [78]:
drop_columns = ['시작일 기준가','종료일 종가','대비','거래량','거래대금']

def create_class_and_merge(data_df, label_df, drop_columns):
    label_df["class"] = label_df["등락률"].apply(
        lambda x: 0 if x <= -10 else (1 if x < 0 else (2 if x < 10 else 3))
    )
    label_df = label_df.drop(drop_columns, axis=1)
    merged_df = pd.merge(data_df, label_df, on=["종목코드"], how="outer")

    return merged_df

In [79]:
import pandas as pd

# 1step : preprocess

BS_item = ["유동자산", "비유동자산", "자산총계", "자본총계", "부채및자본총계", "유동부채", "비유동부채", "부채총계"]
CF_item = ["영업활동현금흐름"]
PL_item = ["매출액", "매출총이익", "순이익"]


def select_non(series):
    return series.dropna().iloc[0] if not series.dropna().empty else None


def preprocess(file_path, item_names_to_extract):
    # Load the data
    data = pd.read_excel(file_path)

    # Erase blank spaces and special characters in '항목명'
    data["항목명"] = (
        data["항목명"].str.replace(" ", "").str.replace(r"[^가-힣]", "", regex=True)
    )

    # Extract the specified item names
    extracted_data = data[data["항목명"].isin(item_names_to_extract)]
    grouped_data = (
        extracted_data.groupby(["회사명", "종목코드", "항목명"])
        .agg({"당기": select_non})
        .reset_index()
    )

    final_data = grouped_data.pivot(
        index=["회사명", "종목코드"], columns="항목명", values="당기"
    ).reset_index()

    return final_data


# 2nd step : fillna preprocessed dataframe


def fillnan(df):
    # Filling NaN values using the provided formulas
    df["자산총계"] = df["자산총계"].fillna(df["유동자산"] + df["비유동자산"])
    df["부채총계"] = df["부채총계"].fillna(df["유동부채"] + df["비유동부채"])
    df["부채및자본총계"] = df["부채및자본총계"].fillna(df["부채총계"] + df["자본총계"])
    df["자본총계"] = df["자본총계"].fillna(df["부채및자본총계"] - df["부채총계"])
    return df


# 3rd step : merge preprocessed dataframe

import pandas as pd


def merge(BS_df, CF_df, IC_df):
    merged_df = pd.merge(BS_df, CF_df, on=["회사명", "종목코드"], how="outer")
    merged_df = pd.merge(merged_df, IC_df, on=["회사명", "종목코드"], how="outer")

    return merged_df


# 4th step : merge label
def create_class_and_merge(data_df, label_df, drop_columns):
    label_df["class"] = label_df["등락률"].apply(
        lambda x: 0 if x <= -10 else (1 if x < 0 else (2 if x < 10 else 3))
    )
    label_df = label_df.drop(drop_columns, axis=1)
    merged_df = pd.merge(data_df, label_df, on=["종목코드"], how="outer")

    return merged_df


filterd_BS_1Q_2020 = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_1Q_BS_filter.xlsx", BS_item)
filterd_CF_1Q_2020 = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_1Q_CF_filter.xlsx", CF_item)
filterd_PL_1Q_2020 = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_1Q_PL_filter.xlsx", PL_item)

filterd_BS_1Q_2020 = fillnan(filterd_BS_1Q_2020)
filterd_CF_1Q_2020 = fillnan(filterd_CF_1Q_2020)
filterd_PL_1Q_2020 = fillnan(filterd_PL_1Q_2020)

df_1Q_2020 = merge(filterd_BS_1Q_2020, filterd_CF_1Q_2020, filterd_PL_1Q_2020)


filterd_BS_2Q_2020 = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_2Q_BS_filter.xlsx", BS_item)
filterd_CF_2Q_2020 = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_2Q_CF_filter.xlsx", CF_item)
filterd_PL_2Q_2020 = preprocess("C:/Users/dochy/Desktop/고려대/딥러닝/project/2020_2Q_PL_filter.xlsx", PL_item)

filterd_BS_2Q_2020 = fillnan(filterd_BS_2Q_2020)
filterd_CF_2Q_2020 = fillnan(filterd_CF_2Q_2020)
filterd_PL_2Q_2020 = fillnan(filterd_PL_2Q_2020)

df_2Q_2020 = merge(filterd_BS_2Q_2020, filterd_CF_2Q_2020, filterd_PL_2Q_2020)

KeyError: '자산총계'

In [83]:
df_2Q_2020

,Unnamed: 0,회사명,종목코드,부채및자본총계,부채총계,비유동부채,비유동자산,유동부채,유동자산,자본총계,자산총계,영업활동현금흐름,매출액,매출총이익,순이익
0,0,3S,060310,5.053320e+10,1.835551e+10,2.685483e+09,3.119785e+10,1.567003e+10,1.933535e+10,3.217769e+10,5.053320e+10,-1.612113e+09,NaN,5.758884e+08,-7.142743e+08
1,1,AJ네트웍스,095570,1.778365e+12,1.442786e+12,6.730463e+11,1.289645e+12,7.697398e+11,4.729384e+11,3.355788e+11,1.778365e+12,-2.673038e+10,3.063528e+11,NaN,2.936957e+09
2,2,AK홀딩스,006840,4.139619e+12,2.933516e+12,1.350699e+12,2.989411e+12,1.582817e+12,1.150209e+12,1.206104e+12,4.139619e+12,-1.801137e+11,5.376024e+11,9.268039e+10,-1.030528e+11
3,3,APS홀딩스,054620,2.963081e+11,8.901188e+10,6.175269e+10,2.517066e+11,2.725919e+10,4.460147e+10,2.072962e+11,2.963081e+11,-2.514411e+09,6.056761e+09,2.717268e+09,-5.497724e+09
4,4,AP시스템,265520,4.320825e+11,3.057251e+11,7.738815e+10,1.595525e+11,2.283369e+11,2.725299e+11,1.263574e+11,4.320825e+11,6.717033e+09,1.285126e+11,2.105618e+10,4.662415e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1705,1705,휴켐스,069260,8.310233e+11,1.589937e+11,8.906189e+10,4.738004e+11,6.993178e+10,3.572229e+11,6.720296e+11,8.310233e+11,4.435421e+10,1.246375e+11,2.405845e+10,1.008381e+10
1706,1706,흥국,010240,1.029121e+11,3.266830e+10,1.044508e+10,4.639592e+10,2.222323e+10,5.651619e+10,7.024381e+10,1.029121e+11,6.128285e+08,2.931115e+10,5.917148e+09,3.204190e+09
1707,1707,흥국에프엔비,189980,9.374722e+10,2.882998e+10,2.015779e+09,5.333368e+10,2.681420e+10,4.041353e+10,6.491724e+10,9.374722e+10,-7.318003e+08,1.402949e+10,5.065893e+09,2.130990e+09
1708,1708,흥아해운,003280,3.674264e+11,3.432330e+11,1.927731e+11,2.912661e+11,1.504599e+11,7.616029e+10,2.419337e+10,3.674264e+11,-1.589750e+10,2.214551e+10,4.493514e+09,-1.625346e+09


In [84]:
df_2Q_2020 = create_class_and_merge(df_2Q_2020,df_2Q_2020_label,drop_columns)
df_1Q_2020 = create_class_and_merge(df_1Q_2020,df_1Q_2020_label,drop_columns)

In [20]:
import pandas as pd

# 1step : preprocess

BS_item = ["유동자산", "비유동자산", "자산총계", "자본총계", "부채및자본총계", "유동부채", "비유동부채", "부채총계"]
CF_item = ["영업활동현금흐름"]
PL_item = ["매출액", "매출총이익", "순이익"]


def select_non(series):
    return series.dropna().iloc[0] if not series.dropna().empty else None


def preprocess(file_path, item_names_to_extract):
    # Load the data
    data = pd.read_excel(file_path)

    # Erase blank spaces and special characters in '항목명'
    data["항목명"] = (
        data["항목명"].str.replace(" ", "").str.replace(r"[^가-힣]", "", regex=True)
    )

    # If '종목코드' column is present, format it by removing brackets
    if "종목코드" in data.columns:
        data["종목코드"] = data["종목코드"].str.replace("[", "").str.replace("]", "")

    # Extract the specified item names
    extracted_data = data[data["항목명"].isin(item_names_to_extract)]
    grouped_data = (
        extracted_data.groupby(["회사명", "종목코드", "항목명"])
        .agg({"당기": select_non})
        .reset_index()
    )

    final_data = grouped_data.pivot(
        index=["회사명", "종목코드"], columns="항목명", values="당기"
    ).reset_index()

    return final_data


# 2nd step : fillna preprocessed dataframe


def fillnan(df):
    # Filling NaN values using the provided formulas
    df["자산총계"] = df["자산총계"].fillna(df["유동자산"] + df["비유동자산"])
    df["부채총계"] = df["부채총계"].fillna(df["유동부채"] + df["비유동부채"])
    df["부채및자본총계"] = df["부채및자본총계"].fillna(df["부채총계"] + df["자본총계"])
    df["자본총계"] = df["자본총계"].fillna(df["부채및자본총계"] - df["부채총계"])
    return df


# 3rd step : merge preprocessed dataframe

import pandas as pd


def merge(BS_df, CF_df, IC_df):
    merged_df = pd.merge(BS_df, CF_df, on=["회사명", "종목코드"], how="outer")
    merged_df = pd.merge(merged_df, IC_df, on=["회사명", "종목코드"], how="outer")

    return merged_df


# 4th step : merge label
def create_class_and_merge(data_df, label_df, drop_columns):
    label_df["class"] = label_df["등락률"].apply(
        lambda x: 0 if x <= -10 else (1 if x < 0 else (2 if x < 10 else 3))
    )
    label_df = label_df.drop(drop_columns, axis=1)
    merged_df = pd.merge(data_df, label_df, on=["종목코드"], how="outer")

    return merged_df


filterd_BS_1Q_2020 = preprocess("2020_1Q_BS_filter.xlsx", BS_item)
filterd_CF_1Q_2020 = preprocess("2020_1Q_CF_filter.xlsx", CF_item)
filterd_PL_1Q_2020 = preprocess("2020_1Q_PL_filter.xlsx", PL_item)

filterd_BS_1Q_2020 = fillnan(filterd_BS_1Q_2020)
filterd_CF_1Q_2020 = fillnan(filterd_CF_1Q_2020)
filterd_PL_1Q_2020 = fillnan(filterd_PL_1Q_2020)

df_1Q_2020 = merge(filterd_BS_1Q_2020, filterd_CF_1Q_2020, filterd_PL_1Q_2020)



filterd_BS_2Q_2020 = preprocess("2020_2Q_BS_filter.xlsx", BS_item)
filterd_CF_2Q_2020 = preprocess("2020_2Q_CF_filter.xlsx", CF_item)
filterd_PL_2Q_2020 = preprocess("2020_2Q_PL_filter.xlsx", PL_item)

filterd_BS_2Q_2020 = fillnan(filterd_BS_2Q_2020)
filterd_CF_2Q_2020 = fillnan(filterd_CF_2Q_2020)
filterd_PL_2Q_2020 = fillnan(filterd_PL_2Q_2020)

df_2Q_2020 = merge(filterd_BS_2Q_2020, filterd_CF_2Q_2020, filterd_PL_2Q_2020)



filterd_BS_1Q_2023 = preprocess("2023_1Q_BS_filter.xlsx", BS_item)
filterd_CF_1Q_2023 = preprocess("2023_1Q_CF_filter.xlsx", CF_item)
filterd_PL_1Q_2023 = preprocess("2023_1Q_PL_filter.xlsx", PL_item)

filterd_BS_1Q_2023 = fillnan(filterd_BS_1Q_2023)
filterd_CF_1Q_2023 = fillnan(filterd_CF_1Q_2023)
filterd_PL_1Q_2023 = fillnan(filterd_PL_1Q_2023)

df_1Q_2023 = merge(filterd_BS_1Q_2023, filterd_CF_1Q_2023, filterd_PL_1Q_2023)


KeyError: "Column(s) ['당기'] do not exist"